In [17]:
import os
import re
import math
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates

from matplotlib.pyplot import figure


import networkx as nx
import seaborn as sns

from wordcloud import WordCloud
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import re
import string
import math
import random
from random import choice, choices
import time
import gc
from IPython.display import display

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from scipy.sparse import coo_matrix
# import implicit

import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    

import os
import gensim
from nltk.tokenize import word_tokenize
import dateutil.parser
from datetime import datetime
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)

%matplotlib inline


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\glin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [5]:
pro= pd.read_csv('professionals.csv')
stu=pd.read_csv("students.csv")
qs=pd.read_csv("questions.csv")
ans= pd.read_csv('answers.csv')
com= pd.read_csv('comments.csv')
email= pd.read_csv('emails.csv')
match= pd.read_csv('matches.csv')
school_mem= pd.read_csv('school_memberships.csv')
tags= pd.read_csv('tags.csv')
qs_tags=pd.read_csv('tag_questions.csv')
user_tag=pd.read_csv('tag_users.csv')

## Build out Recommender - Find Similar Questions

Find similar questions that professionals have answered in the past so students don't need to post same questions. 

In [7]:
qs.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a maths teacher useful? #college #professor #lecture
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I do to become an army officer?,I am Priyanka from Bangalore . Now am in 10th std . When I go to college I should not get confused on what I want to take to become army officer. So I am asking this question #military #army
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase your chances for jobs back home?,"I'm planning on going abroad for my first job. It will be a teaching job and I don't have any serious career ideas. I don't know what job I would be working if I stay home instead so I'm assuming staying or leaving won't makeba huge difference in what I care about, unless I find something before my first job. I can think of ways that going abroad can be seen as good and bad. I do not know which side respectable employers willl side with. #working-abroad #employment- #overseas"
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,"To become a specialist in business management, will I have to network myself?",i hear business management is a hard way to get a job if you're not known in the right areas. #business #networking
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for students that are first generation and live in GA?,I'm trying to find scholarships for first year students but they all seem to be for other states besides GA. Any help??\r\n\r\n#college\r\n#scholarships \r\n#highschoolsenior \r\n#firstgeneration \r\n


In [8]:
tags.head()

,tags_tag_id,tags_tag_name
0,27490,college
1,461,computer-science
2,593,computer-software
3,27292,business
4,18217,doctor


In [9]:
qs_tags.head()

,tag_questions_tag_id,tag_questions_question_id
0,28930,cb43ebee01364c68ac61d347a393ae39
1,28930,47f55e85ce944242a5a347ab85a8ffb4
2,28930,ccc30a033a0f4dfdb2eb987012f25792
3,28930,e30b274e48d741f7bf50eb5e7171a3c0
4,28930,3d22742052df4989b311b4195cbb0f1a


In [31]:
# Concentrate tags to questions

qs_tagnames= pd.merge(qs_tags,tags,left_on='tag_questions_tag_id',right_on='tags_tag_id')
qs_tagnames=qs_tagnames.drop(['tags_tag_id','tag_questions_tag_id'],axis=1)

qs_tag_pivot=qs_tagnames.pivot_table(index='tag_questions_question_id',values='tags_tag_name',aggfunc=lambda x: " ".join(x))
qs_tag_pivot['tag_questions_question_id']=qs_tag_pivot.index

qs_tag_pivot=qs_tag_pivot.reset_index(drop=True)


In [32]:
qs_tag_pivot.head()

,tags_tag_name,tag_questions_question_id
0,internship technology high-school information-technology majoring time,0003e7bf48f24b5c985f8fce96e611f3
1,psychology law,0006609dd4da40dcaa5a83e0499aba14
2,biology marine,000af224bc2f4e94a19f8b62ba279cc4
3,teaching exercise-science school exercise,000b30fb534b41f7b716fa9ebf9c3f35
4,math puremathematics,0018752e44b44e26bb74a0a43232b4d6


In [33]:
# Combine question table with tags

qs_with_tags=pd.merge(qs,qs_tag_pivot,left_on='questions_id',right_on='tag_questions_question_id')
print("Number of questions with tags:",len(qs_with_tags))
qs_with_tags.head(2)

Number of questions with tags: 23288


,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,tags_tag_name,tag_questions_question_id
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a maths teacher useful? #college #professor #lecture,lecture college professor,332a511f1569444485cf7a7a556a5e54
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I do to become an army officer?,I am Priyanka from Bangalore . Now am in 10th std . When I go to college I should not get confused on what I want to take to become army officer. So I am asking this question #military #army,military army,eb80205482e4424cad8f16bc25aa2d9c


In [14]:
# Combine question title, body, and tags

raw_documents=qs_with_tags['questions_title']+qs_with_tags['questions_body']+qs_with_tags['tags_tag_name']
raw_documents.head()

0    Teacher   career   questionWhat  is  a  maths  teacher?   what  is  a  maths  teacher  useful? #college #professor #lecturelecture college professor                                                                                                                                                                                                                                                                                                                                                                                                                                                 
1    I want to become an army officer. What can I do to become an army officer?I am Priyanka from Bangalore . Now am in 10th std . When I go to college I should not get confused on what I want to take to become army officer. So I am asking this question  #military #armymilitary army                                                                                                                                  

### Text Processing

In [34]:
print("Number of Questions:",len(raw_documents))

#Tokenizing data
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]

# Create dictionary
dictionary = gensim.corpora.Dictionary(gen_docs)

# Creat Document-Term Matrix
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

# Creat TF-IDF Model
tf_idf = gensim.models.TfidfModel(corpus)

# Creat Similarity Checker
similar_qs = gensim.similarities.Similarity("",tf_idf[corpus],num_features=len(dictionary))

print("Processing Completed!")

Number of Questions: 23288
Processing Completed!


In [109]:
# New questions posted
# Query='Can I become data scientist without studying at university?#technology #data-science'
# Query = 'Is geology a good major?'
Query = 'How many years in total would it take to become an Emergency Physician?'
Query

'How many years in total would it take to become an Emergency Physician?'

In [110]:
query_doc = [w.lower() for w in word_tokenize(Query)]
query_doc_bow = dictionary.doc2bow(query_doc)
query_doc_tf_idf = tf_idf[query_doc_bow]

In [111]:
q_sim=similar_qs[query_doc_tf_idf]

In [112]:
sim_threshold=0.10

In [113]:
# Display similar questions from the past:

qs_with_tags['Similarity']=q_sim
ques=qs_with_tags[qs_with_tags['Similarity']>=sim_threshold]
ques=ques.sort_values('Similarity',ascending=False)
ques.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,tags_tag_name,tag_questions_question_id,Similarity
136,6eeabcf49fb94f5eae61a9febb21307e,bba963d2034c4b449cd1583f58cfada0,2018-01-12 20:45:53 UTC+0000,How many years in total would it take to become an Emergency Physician?,#medicine #erdoctor #hospital #healthcare,medicine healthcare hospital erdoctor,6eeabcf49fb94f5eae61a9febb21307e,0.465809
5287,5a1d846157254bcf9c3907735f0fbbca,b7bc5589f036447dbb250f300e890780,2018-08-12 19:36:08 UTC+0000,How many years of school to I have to go through in total if i want to become a Pediatrician,How many years beyond a 4-year Bachelor Degree? Like how many years of Med School will I have to go through? #doctor,doctor,5a1d846157254bcf9c3907735f0fbbca,0.361321
21378,a8bb9d19bc5946eda29dcf4f2e0c7545,93f209fd2b354bf5b9a961bb88bcacb9,2018-01-10 18:29:57 UTC+0000,How many years in total after your bachelors do you need to accomplish to become a medical doctor?,"i find the study of medicine very interesting but the amount of time and money required seems a bit much, so i was wondering the amount total needed. #medical-doctor #time-needed",medical-doctor time-needed,a8bb9d19bc5946eda29dcf4f2e0c7545,0.343871
9271,378da016e25a48148cdca5bcfe717e9d,b6a06bb48d034d20bd126debd9481b81,2016-06-22 22:13:22 UTC+0000,What does it take to become a physician?,I am asking because I want to be a physician. #physician,physician,378da016e25a48148cdca5bcfe717e9d,0.328362
921,cf5bd8f5594b42f4a02fa6580948eb6d,b56a0374462f4d7e9d786315828fb57e,2013-08-12 17:26:02 UTC+0000,What are some of the ups and downs of being a emergency nurse?,I am a sophomore who is interested in learning about all the different jobs in the medical field. Is being an emergency nurse fun or difficult? What is a typical day like? What is a bad part? #medicine #nurse #healthcare #emergency,medicine nurse healthcare emergency,cf5bd8f5594b42f4a02fa6580948eb6d,0.310554


## Web Scrapping from Quora

In [116]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
browser = webdriver.Chrome(executable_path='C:/Users/glin/Desktop/YQ/Capstone/data-science-for-good-careervillage/chromedriver_win32/chromedriver.exe')
browser.get("https://www.quora.com/topic/Data-Science")
time.sleep(1)
elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 5
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns-=1
post_elems =browser.find_elements_by_xpath("//a[@class='question_link']")
for post in post_elems:
    print(post.get_attribute("href"))

https://www.quora.com/What-should-I-learn-in-data-science-in-100-hours-I-am-free-for-the-next-10-days-and-would-like-to-learn-whatever-I-can-in-the-next-10-days-and-I-can-put-in-10-hours-a-day-What-can-I-learn-to-get-a-hang-of-it-and-get-started
https://www.quora.com/How-can-I-become-a-data-scientist-1
https://www.quora.com/How-do-I-get-a-job-as-a-data-scientist-if-I-have-no-prior-experience-It-seems-like-Kaggle-is-either-meaningless-or-an-ineffective-route-to-becoming-a-data-scientist-I-was-also-told-there-is-no-point-in-putting-up-my-code-on-GitHub
https://www.quora.com/How-can-I-become-a-data-scientist-1
https://www.quora.com/Why-do-so-many-statisticians-not-want-to-become-data-scientists-Why-are-they-not-interested-in-Big-Data
https://www.quora.com/How-can-you-set-yourself-apart-when-everyone-is-doing-machine-learning-or-data-science-in-2019
https://www.quora.com/As-a-data-scientist-what-career-advice-changed-your-life
https://www.quora.com/As-a-data-scientist-what-tips-would-you-h

In [102]:
# import imporlib
import quoraCrawler2
# importlib.reload(quoraCrawler)
quoraCrawler2.crawlTopicHierarchy()

In crawlTopicHierarchy()...
Current url:http://www.quora.com/Preventive-Medicine?share=1 current depth:0 depth:0
Preventive-Medicine


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
